This notebook has some profiling of Dask used to make a selection along both first and second axes of a large-ish multidimensional array. The use case is making selections of genotype data, e.g., as required for making a web-browser for genotype data as in www.malariagen.net/apps/ag1000g.

In [2]:
import zarr; print('zarr', zarr.__version__)
import dask; print('dask', dask.__version__)
import dask.array as da
import numpy as np

zarr 2.1.4
dask 0.15.0


## Synthetic data

In [15]:
# create a synthetic dataset for profiling
a = zarr.array(np.random.randint(-1, 4, size=(200000, 200, 2), dtype='i1'),
               chunks=(10000, 100, 2), compressor=zarr.Blosc(cname='zstd', clevel=1, shuffle=2))

Array((200000, 200, 2), float32, chunks=(10000, 100, 2), order=C)
  nbytes: 305.2M; nbytes_stored: 348; ratio: 919540.2; initialized: 0/40
  compressor: Blosc(cname='zstd', clevel=1, shuffle=2)
  store: dict

In [25]:
out_name='test'
store = zarr.DirectoryStore(out_name)
the_group=zarr.hierarchy.group(store=store,overwrite=True,
                                    synchronizer=zarr.ThreadSynchronizer())
b = the_group.empty('dna',shape=[200000,200,2], dtype='float32',
               chunks=a.chunks, compressor=zarr.Blosc(cname='zstd', clevel=1, shuffle=2))
b[...]=a[...]

In [26]:
%time d = da.from_array(a, chunks=(a.chunks[0], None, None))
d

CPU times: user 208 ms, sys: 0 ns, total: 208 ms
Wall time: 206 ms


dask.array<array-5..., shape=(20000000, 200, 2), dtype=int8, chunksize=(10000, 200, 2)>

In [23]:
%time ds = d[c][:, s]

NameError: name 'd' is not defined

In [28]:
cProfile.run('d[c][:, s]', sort='time')

         80095589 function calls (60091843 primitive calls) in 19.467 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
10001773/6    4.872    0.000    6.456    1.076 slicing.py:623(check_index)
        2    3.517    1.758    4.357    2.179 slicing.py:398(partition_by_size)
10001775/2    3.354    0.000    6.484    3.242 slicing.py:540(posify_index)
 40007358    2.965    0.000    2.965    0.000 {built-in method builtins.isinstance}
        2    1.749    0.875    6.484    3.242 slicing.py:563(<listcomp>)
        1    0.878    0.878    0.878    0.878 slicing.py:44(<listcomp>)
 10019804    0.451    0.000    0.451    0.000 {built-in method builtins.len}
 10027774    0.392    0.000    0.392    0.000 {method 'append' of 'list' objects}
        2    0.363    0.181    0.363    0.181 slicing.py:420(issorted)
        2    0.270    0.135    4.786    2.393 slicing.py:441(take_sorted)
        1    0.207    0.207    0.207    0.207 {method '

In [29]:
%time ds[1000000:1100000].compute(optimize_graph=False)

CPU times: user 452 ms, sys: 8 ms, total: 460 ms
Wall time: 148 ms


array([[[ 2, -1],
        [ 2,  3],
        [ 3,  0],
        ..., 
        [ 1,  3],
        [-1, -1],
        [ 1,  1]],

       [[ 1, -1],
        [ 2,  2],
        [-1,  2],
        ..., 
        [ 2, -1],
        [ 1,  3],
        [-1, -1]],

       [[ 1, -1],
        [ 2,  0],
        [ 0,  3],
        ..., 
        [ 2,  2],
        [ 3,  2],
        [ 0,  2]],

       ..., 
       [[ 1,  2],
        [ 3, -1],
        [ 2,  1],
        ..., 
        [ 1,  2],
        [ 1,  0],
        [ 2,  0]],

       [[ 1,  2],
        [ 1,  0],
        [ 2,  3],
        ..., 
        [-1,  2],
        [ 3,  3],
        [ 1, -1]],

       [[-1,  3],
        [ 2,  2],
        [ 1,  1],
        ..., 
        [ 3,  3],
        [ 0,  0],
        [ 0,  2]]], dtype=int8)

In [30]:
# problem is in fact just the dim0 selection
cProfile.run('d[c]', sort='time')

         80055494 function calls (60052157 primitive calls) in 19.425 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
10001670/3    5.032    0.000    6.671    2.224 slicing.py:623(check_index)
        1    3.459    3.459    4.272    4.272 slicing.py:398(partition_by_size)
10001671/1    3.287    0.000    6.378    6.378 slicing.py:540(posify_index)
 40006704    2.999    0.000    2.999    0.000 {built-in method builtins.isinstance}
        1    1.731    1.731    6.378    6.378 slicing.py:563(<listcomp>)
        1    0.849    0.849    0.849    0.849 slicing.py:44(<listcomp>)
 10011685    0.433    0.000    0.433    0.000 {built-in method builtins.len}
 10015670    0.381    0.000    0.381    0.000 {method 'append' of 'list' objects}
        1    0.355    0.355    0.355    0.355 slicing.py:420(issorted)
        1    0.196    0.196    0.196    0.196 {method 'tolist' of 'numpy.ndarray' objects}
        1    0.193    0.193    0.193  